
0. Install and Import Dependencies

In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [6]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv 

In [7]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions


1. Make Some Detections

In [9]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [10]:
results.face_landmarks.landmark[0].visibility

0.0


2. Capture Landmarks & Export to CSV

In [11]:
import csv
import os
import numpy as np

In [12]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [53]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [54]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [59]:
class_name = "Greetings"

In [60]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


3. Train Custom Model Using Scikit Learn


3.1 Read in Collected Data and Process

In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [62]:
df = pd.read_csv('coords.csv')

In [63]:
df[df['class']=="Sad"]

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Sad,0.515912,0.327699,-1.180751,0.999871,0.547356,0.271942,-1.144316,0.999882,0.565439,...,-0.010937,0.0,0.582452,0.284713,0.001509,0.0,0.588377,0.276495,0.001489,0.0
1,Sad,0.513921,0.333449,-1.135985,0.999858,0.545993,0.274086,-1.101505,0.999869,0.565188,...,-0.010020,0.0,0.580053,0.280911,0.003963,0.0,0.585523,0.275561,0.003669,0.0
2,Sad,0.512179,0.336637,-1.110512,0.999852,0.544572,0.275316,-1.076720,0.999864,0.564706,...,-0.010986,0.0,0.579401,0.279346,0.002967,0.0,0.584722,0.274277,0.002604,0.0
3,Sad,0.510677,0.338504,-1.168302,0.999836,0.543580,0.276034,-1.132628,0.999851,0.564398,...,-0.011556,0.0,0.579564,0.278963,0.002561,0.0,0.584818,0.274160,0.002191,0.0
4,Sad,0.510086,0.339934,-1.126713,0.999841,0.543109,0.276495,-1.093187,0.999853,0.564265,...,-0.011445,0.0,0.579565,0.277974,0.002918,0.0,0.584708,0.273275,0.002548,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Sad,0.536477,0.362120,-0.715767,0.999219,0.559647,0.312436,-0.682964,0.998836,0.576781,...,-0.009472,0.0,0.591363,0.310241,-0.000145,0.0,0.595322,0.304981,-0.000559,0.0
153,Sad,0.536937,0.362239,-0.726643,0.999170,0.560062,0.312483,-0.696538,0.998757,0.577196,...,-0.009380,0.0,0.591172,0.310362,-0.000087,0.0,0.595128,0.305064,-0.000515,0.0
154,Sad,0.537136,0.362249,-0.701985,0.999181,0.560302,0.312490,-0.674216,0.998771,0.577426,...,-0.008988,0.0,0.591304,0.311462,-0.000047,0.0,0.595358,0.305590,-0.000411,0.0
155,Sad,0.537148,0.362256,-0.706337,0.999152,0.560379,0.312502,-0.677403,0.998732,0.577485,...,-0.008691,0.0,0.589902,0.313755,-0.000270,0.0,0.593994,0.307157,-0.000543,0.0


In [64]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

3.2 Train Machine Learning Classification Model

In [66]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [67]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [68]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

c:\Users\FABDELLI\python\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [69]:
fit_models['rc'].predict(X_test)

array(['Sad', 'Greetings', 'Happy', 'Greetings', 'Greetings', 'Greetings',
       'Greetings', 'Happy', 'Greetings', 'Greetings', 'Sad', 'Sad',
       'Happy', 'Greetings', 'Greetings', 'Happy', 'Greetings', 'Sad',
       'Greetings', 'Greetings', 'Greetings', 'Happy', 'Sad', 'Sad',
       'Happy', 'Greetings', 'Sad', 'Sad', 'Sad', 'Greetings', 'Happy',
       'Sad', 'Greetings', 'Happy', 'Happy', 'Sad', 'Greetings', 'Happy',
       'Greetings', 'Sad', 'Sad', 'Sad', 'Sad', 'Greetings', 'Sad',
       'Greetings', 'Happy', 'Sad', 'Happy', 'Happy', 'Happy', 'Sad',
       'Happy', 'Sad', 'Greetings', 'Happy', 'Greetings', 'Happy',
       'Happy', 'Sad', 'Happy', 'Sad', 'Happy', 'Sad', 'Sad', 'Greetings',
       'Sad', 'Greetings', 'Greetings', 'Sad', 'Happy', 'Happy', 'Sad',
       'Greetings', 'Sad', 'Happy', 'Sad', 'Greetings', 'Happy',
       'Greetings', 'Happy', 'Happy', 'Happy', 'Sad', 'Greetings', 'Sad',
       'Sad', 'Greetings', 'Greetings', 'Happy', 'Sad', 'Greetings',
       'Sa

3.3 Evaluate and Serialize Model

In [70]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle

In [71]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [72]:
fit_models['rf'].predict(X_test)

array(['Sad', 'Greetings', 'Happy', 'Greetings', 'Greetings', 'Greetings',
       'Greetings', 'Happy', 'Greetings', 'Greetings', 'Sad', 'Sad',
       'Happy', 'Greetings', 'Greetings', 'Happy', 'Greetings', 'Sad',
       'Greetings', 'Greetings', 'Greetings', 'Happy', 'Sad', 'Sad',
       'Happy', 'Greetings', 'Sad', 'Sad', 'Sad', 'Greetings', 'Happy',
       'Sad', 'Greetings', 'Happy', 'Happy', 'Sad', 'Greetings', 'Happy',
       'Greetings', 'Sad', 'Sad', 'Sad', 'Sad', 'Greetings', 'Sad',
       'Greetings', 'Happy', 'Sad', 'Happy', 'Happy', 'Happy', 'Sad',
       'Happy', 'Sad', 'Greetings', 'Happy', 'Greetings', 'Happy',
       'Happy', 'Sad', 'Happy', 'Sad', 'Happy', 'Sad', 'Sad', 'Greetings',
       'Sad', 'Greetings', 'Greetings', 'Sad', 'Happy', 'Happy', 'Sad',
       'Greetings', 'Sad', 'Happy', 'Sad', 'Greetings', 'Happy',
       'Greetings', 'Happy', 'Happy', 'Happy', 'Sad', 'Greetings', 'Sad',
       'Sad', 'Greetings', 'Greetings', 'Happy', 'Sad', 'Greetings',
       'Sa

In [73]:
y_test

7            Sad
412    Greetings
254        Happy
308    Greetings
289    Greetings
         ...    
90           Sad
400    Greetings
54           Sad
323    Greetings
148          Sad
Name: class, Length: 129, dtype: object

In [74]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)


4. Make Detections with Model

In [75]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [76]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [ ]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row


            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [79]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))

(373, 150)

In [ ]:
# Git steps to push code form beginning

git init
git add file
git commit -m "first commit"
git remote add origin "http...."
git push -u origin master

# Git steps to push code in new branch collaborating with team
git pull
git status
git checkout -b "name of branch"
git config --global user.email farah.abdelli@esprit.tn
git config --global user.name farahabdelli
git commit "commit"
git push -u origin "branch name"


#commands
python -m pip install 'package'
python -m pip install --upgrade pip
streamlit run datavis.py